MINIMISE 
$\\f_1(x) = 10x_1 + 2x_2\\$
$f_2(x) = 3x_1 ^ 2 + 2x_2 ^ 2$ 

SUBJECT TO
$\\ x_1 \geq 2\\$
$ x_2 \geq 3 \\$
$ x_1 + x_2 \leq 12$

Both f_1 and f_2 are continously differentiable 


Aim of problem
min f(x) sub to contraints
Quasi-Newton methods
B_n approx = nabla ^2 f(x_n)



In [ ]:
#TODO :UPDATE CALC AS INDICATED IN ORIGINAL PAPER
import numpy as np
from scipy.optimize import minimize

def search_dir(x, grads, B,n_args, printRes = False):
    """
    Calcualtes the search direction of a function F from its gradient,
    and hessian approximations.

    INPUT: x - the current point
           grads - a list of gradient functions
           B - a list of hessian approximations TODO:check
           n_args - the number of arguments of the function
           printRes - a boolean to print the result
    OUTPUT: dOpt - the optimal search direction
            thetaOpt - the optimal value of the objective function
    """
    
    #Calculate d(x) = arg min d(x) {max_i {grad_i(x)^T d + 0.5 d^T B_i d}}
    def obj(d,B,grads, x):
        #Function to maximise
        return np.max([(grad(x)).T @ d + 0.5 * d.T @ B_j @ d for grad, B_j in zip(grads, B)])
    
    #Initial guess for d
    d0 = np.zeros(n_args)
    result = minimize(obj, d0, args=(B, grads,x))
    
    dOpt = result.x

    #Calculate theta(x) = max_i {grad_i(x)^T d + 0.5 d^T B_i d}
    thetaOpt = obj(dOpt, B, grads, x)

    return dOpt, thetaOpt



In [3]:
def Delta(grads,x,d):
    return np.max([grad(x).T @ d for grad in grads])

In [ ]:
#TODO: add tolerance?
def Armijo(F,grads, x, alpha, d, c1):
    for f in F:
        if f(x + alpha * d) > f(x) + c1 * alpha * Delta(grads, x, d):
            return False
    return True

In [ ]:
#TODO: add tolerance ?
def Curvature(F, grads, x, alpha, d, c2):
    if Delta(grads,x + alpha * d,d) >= c2 * Delta(grads, x,d):
        return True
    return False

In [ ]:
def WLinesearch(F,grads, x,d,c1 = 0.1, c2 = 0.9):
    """"
    Linesearch
    Inputs: F - a list of functions
            grads - a list of gradient functions
            x - the current point
            d - the search direction
            c1 - the Armijo condition parameter
            c2 - the curvature condition parameter
    Outputs: alpha - the step size
    """
    alpha = 1
    alpha_bar = 0
    alpha_hat = 0
    armijo_test = Armijo(F, grads, x, alpha, d, c1)
    curvature_test = Curvature(F, grads, x, alpha, d, c2)

    while not armijo_test or not curvature_test:
        if not armijo_test:
            alpha_bar = alpha
            alpha = (alpha_bar + alpha_hat) / 2
        elif not curvature_test:
            alpha_bar = alpha

            if alpha_hat == 0:
                alpha = 2 * alpha_bar
            else:
                alpha = (alpha_bar + alpha_hat) / 2
        
        armijo_test = Armijo(F, grads, x, alpha, d, c1)
        curvature_test = Curvature(F, grads, x, alpha, d, c2)

    return alpha

In [ ]:
import array

def BUpdate(F, grads,B0, x0, x1):
    #Prep
    grads_x0 = np.array([grad(x0) for grad in grads])
    grads_x1 = np.array([grad(x1) for grad in grads])

    y = grads_x1 - grads_x0
    s = x1 - x0
    sy = s.T @ y
    #rho
    if sy> 0:
        rho_over = 1/sy
    else:
        rho_over = 1/ (Delta(grads,x1,s) - grads_x0.T @ s)

    
    denom = 1/((rho_over - s.T @ y)**2 + rho_over * s.T @ B0 @ s)

    #Update B
    t1 = rho_over * B0 @ s @ s.T @ B0
    t2 = (s.T @ B0 @ s) * (y @ y.T)
    t3 = (rho_over - s.T@ y) * (y @ s.T @ B0 + B0 @ s @ y.T)
          
    B1 = B0  + denom * (-t1 + t2 + t3)
    return B1
    

In [ ]:
def BFGS(F,grads, x, B, c1, c2, iter = 100, tol = 1e-6):

    n_args = len(x)
    x0 = x
    Bn = B
    for i in range(iter):

        #STEP 1: Search direction
        d,theta = search_dir(x0, grads, Bn, n_args)

        #STEP 2: Stopping criterion
        if abs(theta) < tol:
            return x
        
        #STEP 3: Line search
        alpha = WLinesearch(F, grads, x, d, c1, c2)
        xi = x + alpha * d

        sk = xi - x

        Bn = BUpdate(F, grads, Bn, x0, xi)
    
    print("BFGS did not converge")
    return xi
